In [1]:
import torch
import torch.nn.functional as F
from src.layer import TransformerBlock, HierarchicalTransformerBlock

%load_ext autoreload
%autoreload 2

In [ ]:
768 * 4 * 2/3
# d_ff = d_model * 4 * 2/3

In [ ]:
768 / 8
# d_k = d_model / h

#### test transformer block

In [ ]:
encoder = TransformerBlock(
    d_model=768,
    d_ff=2048,
    h=12,
    dropout=0,
    norm_type="layer",
    ffn_type="swiglu",
)

In [ ]:
encoder = encoder.to("cuda")
encoder

In [ ]:
inputs = torch.randn(2, 128, 512).to("cuda")
masks = torch.ones(2, 128).to("cuda")

In [ ]:
encoder(inputs, masks).shape

#### test hierarchical transformer block

In [104]:
hat_block = HierarchicalTransformerBlock(
    d_model=512,
    d_ff=1024,
    h=4,
    dropout=0,
    norm_type="layer",
    ffn_type="swiglu",
).cuda()

In [105]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(hat_block) / 1e6

5.250048

In [106]:
(2e4 * 512 + 5.25e6 * 6) / 1e6


41.74

In [3]:
seg_hidden_state = torch.randn(32, 32, 128, 768).cuda()
token_mask = torch.rand(32, 32, 128).bernoulli().cuda()
seg_mask = torch.rand(32, 32).bernoulli().cuda()


In [4]:
hat_block(seg_hidden_state, token_mask, seg_mask).shape

torch.Size([32, 32, 128, 768])

#### RoPE

In [ ]:
from src.layer import RoPE

rope = RoPE(d=256)
x = torch.randn(2, 12, 32, 256)

In [ ]:
rope(x).shape

### test model

In [2]:
from src.fm import FMConfig, FMBase

In [ ]:
cfg = FMConfig(d_model=512, n_heads=8, d_ff=1024, **{"lr": 1e-5, "batch_size": 8})
model = FMBase(cfg).to("cuda")
cfg.to_diff_dict()

AssertionError: d_model must be divisible by h

In [100]:
cfg.lr

1e-05

#### simcse

In [31]:
from src.loss import SimCSE

In [ ]:
model.en

In [ ]:
simcse = SimCSE(model, temperature=0.05)